In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [4]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [5]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [6]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [7]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [8]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [9]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [10]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [11]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [12]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [13]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상', 'path']]

In [14]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# object 형식의 열들에 대해 Label Encoding 수행
for column in df_raw_x.columns:
    if df_raw_x[column].dtype == 'object':
        df_raw_x[column] = label_encoder.fit_transform(df_raw_x[column])

from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# 데이터 스케일링
X_scaled = scaler.fit_transform(df_raw_x)

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    X_scaled, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

print("분할 전 설명변수 데이터 :", df_raw_x.shape)
print("분할 후 설명변수 데이터 :Train", df_train_x.shape, "  Test",df_test_x.shape)

분할 전 설명변수 데이터 : (1693, 25)
분할 후 설명변수 데이터 :Train (1185, 25)   Test (508, 25)


In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 준비
# X와 y는 특성과 타겟 데이터입니다. 이에 맞게 데이터를 로드하고 전처리합니다.
# 예시로는 적절한 데이터를 사용하여야 합니다.

# 데이터 전처리
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_raw_x)  # 특성 스케일링

X_train, X_test, y_train, y_test = train_test_split(X_scaled, df_raw_y,
                                                    test_size=0.3, random_state=1234, stratify=df_raw_y )  # 훈련/검증 데이터 분할

# 모델 구성
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # 입력 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(64, activation='relu'),  # 은닉 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(1, activation='sigmoid')  # 출력 레이어 (이진 분류이므로 시그모이드 활성화 함수 사용)
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # 이진 분류이므로 binary crossentropy 사용
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6994 - loss: 0.5996 - val_accuracy: 0.9367 - val_loss: 0.2451
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9226 - loss: 0.2892 - val_accuracy: 0.9367 - val_loss: 0.2242
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9298 - loss: 0.2683 - val_accuracy: 0.9367 - val_loss: 0.2150
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9269 - loss: 0.2359 - val_accuracy: 0.9367 - val_loss: 0.2092
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9164 - loss: 0.2494 - val_accuracy: 0.9367 - val_loss: 0.2056
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9162 - loss: 0.2788 - val_accuracy: 0.9367 - val_loss: 0.2028
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9338 - loss: 0.2192 - val_accuracy: 0.9367 - val_loss: 0.2015
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9030 - loss: 0.2633 - val_accuracy: 0.9409

In [26]:
from sklearn.metrics import confusion_matrix, classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)

# 혼동 행렬 계산
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))

# 분류 보고서 출력
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Confusion Matrix:
[[467   5]
 [ 25  11]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       472
         1.0       0.69      0.31      0.42        36

    accuracy                           0.94       508
   macro avg       0.82      0.65      0.70       508
weighted avg       0.93      0.94      0.93       508



In [27]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [30]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # 입력 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(64, activation='relu'),  # 은닉 레이어
    Dropout(0.5),  # 드롭아웃 레이어
    Dense(1, activation='sigmoid')  # 출력 레이어 (이진 분류이므로 시그모이드 활성화 함수 사용)
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # 이진 분류이므로 binary crossentropy 사용
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_resampled, y_resampled, epochs=300, batch_size=32, validation_split=0.2, verbose=1)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5816 - loss: 0.6815 - val_accuracy: 0.4955 - val_loss: 0.7316
Epoch 2/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7257 - loss: 0.5561 - val_accuracy: 0.5295 - val_loss: 0.6738
Epoch 3/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7354 - loss: 0.5101 - val_accuracy: 0.6432 - val_loss: 0.5499
Epoch 4/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7588 - loss: 0.4633 - val_accuracy: 0.6750 - val_loss: 0.5073
Epoch 5/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7775 - loss: 0.4270 - val_accuracy: 0.7432 - val_loss: 0.4560
Epoch 6/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8059 - loss: 0.4010 - val_accuracy: 0.7886 - val_loss: 0.4130
Epoch 7/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8506 - loss: 0.3409 - val_accuracy: 0.8341 - val_loss: 0.3956
Epoch 8/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8378 - loss: 0.3422 - val_accuracy: 0.8568 - 

In [31]:
from sklearn.metrics import confusion_matrix, classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)

# 혼동 행렬 계산
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))

# 분류 보고서 출력
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Confusion Matrix:
[[466   6]
 [ 27   9]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       472
         1.0       0.60      0.25      0.35        36

    accuracy                           0.94       508
   macro avg       0.77      0.62      0.66       508
weighted avg       0.92      0.94      0.92       508

